# Case Study - 1 (Recruit Restaurant Visitor Forecasting)



### By Junaid Khan (junaidkhangec@gmail.com) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = ""

# Feature Engineering 

In [ ]:
from google.colab import files
def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn import model_selection, ensemble, neighbors
from sklearn.linear_model import LogisticRegression

from subprocess import check_output
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn import preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse.linalg import svds
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import xgboost as xgb
#from yellowbrick.model_selection import RFECV
from sklearn.pipeline import Pipeline
plt.style.use('fivethirtyeight')
import os

In [ ]:
# reading all files
air_reserve = pd.read_csv( root_path + 'air_reserve.csv')
air_store_info = pd.read_csv( root_path + 'air_store_info.csv')
air_visit_data = pd.read_csv( root_path + 'air_visit_data.csv')
hpg_reserve = pd.read_csv(root_path +'hpg_reserve.csv')
hpg_store_info = pd.read_csv( root_path+'hpg_store_info.csv')
store_id_relation = pd.read_csv( root_path+'store_id_relation.csv')
date_info = pd.read_csv(root_path+'date_info.csv')
sample_submission = pd.read_csv(root_path +'sample_submission.csv')

In [ ]:
# setup train & test
test = sample_submission.copy()
test['air_store_id'] = test['id'].apply(lambda x: str(x)[:-11])
test['visit_date'] = test['id'].apply(lambda x: str(x)[-10:])
test = test.drop(['id', 'visitors'], axis = 1)
train = air_visit_data.copy()

## 1.Processing Reservation data

In [ ]:
# combine air and hpg data
hpg_air_reserve = store_id_relation.join(hpg_reserve.set_index('hpg_store_id'), on = 'hpg_store_id')
hpg_air_reserve = hpg_air_reserve.drop('hpg_store_id', axis = 1)
reserve = pd.concat([air_reserve, hpg_air_reserve])

# convert columns of "reserve" table into datetime format
reserve['visit_datetime'] =  pd.to_datetime(reserve['visit_datetime'])
reserve['reserve_datetime'] =  pd.to_datetime(reserve['reserve_datetime'])

# create column for visit date inside "reserve" table
reserve['visit_date'] = reserve['visit_datetime'].apply(lambda x: str(x)[0:10])

# calculate the gap between visit time and reservation time inside "reserve" table
reserve['hour_gap'] = reserve['visit_datetime'].sub(reserve['reserve_datetime'])
reserve['hour_gap'] = reserve['hour_gap'].apply(lambda x: x/np.timedelta64(1,'h'))

# separate reservation into 5 categories based on gap lenght
reserve['reserve_-12_h'] = np.where(reserve['hour_gap'] <= 12,
                                    reserve['reserve_visitors'], 0)
reserve['reserve_12_36_h'] = np.where((reserve['hour_gap'] <= 36) & (reserve['hour_gap'] > 12),
                                       reserve['reserve_visitors'], 0)
reserve['reserve_36_60_h'] = np.where((reserve['hour_gap'] <= 60) & (reserve['hour_gap'] > 36),
                                       reserve['reserve_visitors'], 0)
reserve['reserve_60_84_h'] = np.where((reserve['hour_gap'] <= 84) & (reserve['hour_gap'] > 60),
                                       reserve['reserve_visitors'], 0)
reserve['reserve_84+_h'] = np.where((reserve['hour_gap'] > 84),
                                     reserve['reserve_visitors'], 0)

#reserve['reserve_84+_h'] = np.where((reserve['hour_gap'] <= 960) & (reserve['hour_gap'] > 84),
#                                       reserve['reserve_visitors'], 0)
#reserve['reserve_40+_d'] = np.where((reserve['hour_gap'] > 960),
#                                     reserve['reserve_visitors'], 0)

# group by air_store_id and visit_date
group_list = ['air_store_id', 'visit_date', 'reserve_visitors', 'reserve_-12_h',
              'reserve_12_36_h', 'reserve_36_60_h', 'reserve_60_84_h', 'reserve_84+_h']
#              ,'reserve_40+_d']
reserve = reserve[group_list].groupby(['air_store_id', 'visit_date'], as_index = False).sum()
    
reserve.head(3)

,air_store_id,visit_date,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h
0,air_00a91d42b08b08d9,2016-01-14,2,0,0,0,2,0
1,air_00a91d42b08b08d9,2016-01-15,4,0,0,0,0,4
2,air_00a91d42b08b08d9,2016-01-16,2,0,0,0,2,0


## 2.Genre and Area wise restaurants grouping

In [ ]:
air_genres_area = air_store_info.copy()
air_genres_area = air_genres_area[['air_store_id', 'air_genre_name', 'air_area_name']].groupby(['air_genre_name', 'air_area_name'],
                                                                                              as_index = False).count()
air_genres_area = air_genres_area.rename(columns = {'air_store_id': 'genre_in_area'})

air_genres_area.head(3)

,air_genre_name,air_area_name,genre_in_area
0,Asian,Tōkyō-to Shibuya-ku Shibuya,2
1,Bar/Cocktail,Fukuoka-ken Fukuoka-shi Daimyō,7
2,Bar/Cocktail,Fukuoka-ken Fukuoka-shi Hakata Ekimae,2


## 3.Area wise restaurants grouping 

In [ ]:
# total amount of restaurants in area
air_area = air_store_info.copy()
air_area = air_area[['air_store_id', 'air_area_name']].groupby(['air_area_name'], as_index = False).count()
air_area = air_area.rename(columns = {'air_store_id': 'total_r_in_area'})

air_area.head(3)

,air_area_name,total_r_in_area
0,Fukuoka-ken Fukuoka-shi Daimyō,64
1,Fukuoka-ken Fukuoka-shi Hakata Ekimae,16
2,Fukuoka-ken Fukuoka-shi Imaizumi,2


## 4.Processing weekend and holidays data

In [ ]:
# additional features for weekends and holidays
date_info_mod = date_info.copy()
date_info_mod['holiday_eve'] = np.zeros(date_info_mod.shape[0])
date_info_mod['holiday_eve'].iloc[:-1] = date_info_mod['holiday_flg'].copy().values[1:]
date_info_mod['non_working'] = np.where(date_info_mod['day_of_week'].isin(['Saturday', 'Sunday']) |
                                        date_info_mod['holiday_flg'] == 1, 1, 0)
date_info_mod = date_info_mod.drop('holiday_flg', axis = 1)

date_info_mod.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,calendar_date,day_of_week,holiday_eve,non_working
0,2016-01-01,Friday,1.0,1
1,2016-01-02,Saturday,1.0,1
2,2016-01-03,Sunday,0.0,1


## 5.Visitors statistics on the basis of working & non-working days

In [ ]:
# average visitors per restaurant by working and non-working days
air_visit_wd = air_visit_data.join(date_info_mod.set_index('calendar_date'), on = 'visit_date')
air_visit_wd['visitors'] = air_visit_wd['visitors'] 

# average visitors per restaurant
mean_df = air_visit_wd[['visitors',
                        'air_store_id',
                        'non_working']].copy().groupby(['air_store_id',
                                                        'non_working'],
                                                        as_index = False).mean()
mean_df = mean_df.rename(columns = {'visitors': 'visitors_mean'})


# median visitors per restaurant
median_df = air_visit_wd[['visitors',
                          'air_store_id',
                          'non_working']].copy().groupby(['air_store_id',
                                                          'non_working'],
                                                          as_index = False).median()
median_df = median_df.rename(columns = {'visitors': 'visitors_median'})

# max visitors per restaurant
max_df = air_visit_wd[['visitors',
                       'air_store_id',
                       'non_working']].copy().groupby(['air_store_id',
                                                       'non_working'],
                                                       as_index = False).max()
max_df = max_df.rename(columns = {'visitors': 'visitors_max'})

# min visitors per restaurant
min_df = air_visit_wd[['visitors',
                       'air_store_id',
                       'non_working']].copy().groupby(['air_store_id',
                                                       'non_working'],
                                                       as_index = False).min()
min_df = min_df.rename(columns = {'visitors': 'visitors_min'})

mean_df.head(3)

,air_store_id,non_working,visitors_mean
0,air_00a91d42b08b08d9,0,28.432292
1,air_00a91d42b08b08d9,1,14.800000
2,air_0164b9927d20bcc3,0,9.756098


## 6.Merging with train and test

In [ ]:
# add month of visit
train['month'] = train['visit_date'].apply(lambda x: float(str(x)[5:7]))

# add weekday and holiday flag
train = train.join(date_info_mod.set_index('calendar_date'), on = 'visit_date')
# add genre and area name
train = train.join(air_store_info.set_index('air_store_id'), on = 'air_store_id')
# add quantity of same genre in area
train = pd.merge(train, air_genres_area, how = 'left',
                  left_on = ['air_genre_name', 'air_area_name'],
                  right_on = ['air_genre_name', 'air_area_name'])
# add total quatity of restaurants in area
train = pd.merge(train, air_area, how = 'left',
                  left_on = ['air_area_name'],
                  right_on = ['air_area_name'])

# add reservation information
train = pd.merge(train, reserve, how = 'left',
                  left_on = ['air_store_id', 'visit_date'],
                  right_on = ['air_store_id', 'visit_date'])
# add visitors number mean, median, max and min per each restaurant
train = pd.merge(train, mean_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
train = pd.merge(train, median_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
train = pd.merge(train, max_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
train = pd.merge(train, min_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
# change NaN to 0
train = train.fillna(0) 

In [ ]:
pd.options.display.max_columns = None
train.head(3)

,air_store_id,visit_date,visitors,month,day_of_week,holiday_eve,non_working,air_genre_name,air_area_name,latitude,longitude,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min
0,air_ba937bf13d40fb24,2016-01-13,25,1.0,Wednesday,0.0,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2
1,air_ba937bf13d40fb24,2016-01-14,32,1.0,Thursday,0.0,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2
2,air_ba937bf13d40fb24,2016-01-15,29,1.0,Friday,0.0,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2


In [ ]:
test.head(3)

,air_store_id,visit_date
0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9,2017-04-25


In [ ]:
date_info_mod.head(3)

,calendar_date,day_of_week,holiday_eve,non_working
0,2016-01-01,Friday,1.0,1
1,2016-01-02,Saturday,1.0,1
2,2016-01-03,Sunday,0.0,1


In [ ]:
# add month of visit
test['month'] = test['visit_date'].apply(lambda x: float(str(x)[5:7]))

# add weekday and holiday flag
test = test.join(date_info_mod.set_index('calendar_date'), on = 'visit_date')
# add genre and area name
test = test.join(air_store_info.set_index('air_store_id'), on = 'air_store_id')
# add quantity of same genre in area
test = pd.merge(test, air_genres_area, how = 'left',
                  left_on = ['air_genre_name', 'air_area_name'],
                  right_on = ['air_genre_name', 'air_area_name'])
# add total quatity of restaurants in area
test = pd.merge(test, air_area, how = 'left',
                  left_on = ['air_area_name'],
                  right_on = ['air_area_name'])

# add reservation information
test = pd.merge(test, reserve, how = 'left',
                  left_on = ['air_store_id', 'visit_date'],
                  right_on = ['air_store_id', 'visit_date'])
# add visitors number mean, median, max and min per each restaurant
test = pd.merge(test, mean_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
test = pd.merge(test, median_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
test = pd.merge(test, max_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
test = pd.merge(test, min_df, how = 'left',
                  left_on = ['air_store_id', 'non_working'],
                  right_on = ['air_store_id', 'non_working'])
# change NaN to 0
test = test.fillna(0) 

test.head(3)

,air_store_id,visit_date,month,day_of_week,holiday_eve,non_working,air_genre_name,air_area_name,latitude,longitude,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min
0,air_00a91d42b08b08d9,2017-04-23,4.0,Sunday,0.0,1,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0
1,air_00a91d42b08b08d9,2017-04-24,4.0,Monday,0.0,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0
2,air_00a91d42b08b08d9,2017-04-25,4.0,Tuesday,0.0,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0


In [ ]:
train['id'] = train['air_store_id'] + str('_') + train['visit_date']
test['id'] = test['air_store_id'] + str('_') + test['visit_date']

# getting unimportant columns
drop_train_col = ['air_store_id', u'visit_date', u'visitors', u'month', u'day_of_week',u'air_genre_name', u'air_area_name', u'latitude', u'longitude']
drop_test_col = ['air_store_id', u'visit_date',  u'month', u'day_of_week',u'air_genre_name', u'air_area_name', u'latitude', u'longitude']

train = train.drop(drop_train_col,axis=1)
test = test.drop(drop_test_col,axis=1)

sample_submission['air_store_id'] = sample_submission['id'].apply(lambda x: x.split('_')[0]+str('_') +x.split('_')[1])
sample_submission['visit_date'] = sample_submission['id'].apply(lambda x: x.split('_')[2])
sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'],format= '%Y-%m-%d %H:%M:%S')

air_visit_data = air_visit_data.merge(air_store_info,how = 'left',on= 'air_store_id')
sample_submission = sample_submission.merge(air_store_info,how = 'left',on= 'air_store_id')

air_visit_data['id'] = air_visit_data['air_store_id'] + str('_') + air_visit_data['visit_date']
air_visit_data = air_visit_data.merge(train,on='id',how= 'left')
air_visit_data = air_visit_data.drop('id',axis=1)
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'],format= '%Y-%m-%d')
sample_submission =  sample_submission.merge(test,how = 'left',on='id')

In [ ]:
train.head(3)

,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,id
0,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,air_ba937bf13d40fb24_2016-01-13
1,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,air_ba937bf13d40fb24_2016-01-14
2,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,air_ba937bf13d40fb24_2016-01-15


In [ ]:
test.head(3)

,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,id
0,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0,air_00a91d42b08b08d9_2017-04-23
1,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,air_00a91d42b08b08d9_2017-04-24
2,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,air_00a91d42b08b08d9_2017-04-25


In [ ]:
sample_submission.head(3)

,id,visitors,air_store_id,visit_date,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0


## 7.More features using date

In [ ]:
air_visit_data['visit_date_month'] =air_visit_data.visit_date.dt.month
air_visit_data['visit_date_dayofw'] =air_visit_data.visit_date.dt.dayofweek
air_visit_data['visit_date_year'] =air_visit_data.visit_date.dt.year
air_visit_data['visit_date_dayofm'] =air_visit_data.visit_date.dt.day
air_visit_data['weekofyear'] =air_visit_data.visit_date.dt.weekofyear
sample_submission['visit_date_month'] =sample_submission.visit_date.dt.month
sample_submission['visit_date_dayofw'] =sample_submission.visit_date.dt.dayofweek
sample_submission['visit_date_year'] =sample_submission.visit_date.dt.year
sample_submission['visit_date_dayofm'] =sample_submission.visit_date.dt.day
sample_submission['weekofyear'] =sample_submission.visit_date.dt.weekofyear
air_visit_data.loc[air_visit_data.weekofyear==53,'weekofyear'] =0 
sample_submission.loc[sample_submission.weekofyear==53,'weekofyear'] =0 

sample_submission.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.


,id,visitors,air_store_id,visit_date,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0,4,6,2017,23,16
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,0,2017,24,17
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,1,2017,25,17


## 8.Number of open restaurants on weekly basis

In [ ]:
total_air_ids = list(air_store_info.air_store_id.unique())
df_total = pd.concat((air_visit_data, sample_submission))
df_total = df_total[['weekofyear','visit_date_year','air_store_id']]
df_total = df_total.sort_values(['visit_date_year','weekofyear'])
weekofyear = list(df_total['weekofyear'].unique())
year = list(df_total['visit_date_year'].unique())

In [ ]:
# calculating number of open restaurants on weekly basis
week_open_restr = []
for i in year:
    for j in weekofyear:
        l= len(list(df_total.loc[(df_total.visit_date_year ==i) & (df_total.weekofyear ==j),'air_store_id'].unique()))
        if (i==2017) & (j>22):
            break
        week_open_restr.append([i,j,l])

df_open_restro = pd.DataFrame(week_open_restr)
df_open_restro.columns = ['visit_date_year','weekofyear','no_open_restr']

df_open_restro.head(3)

,visit_date_year,weekofyear,no_open_restr
0,2016,0,99
1,2016,1,293
2,2016,2,297


## 9.Daily, Weekly, Monthly & Yearly visitors statistics

In [ ]:
columns = [i for i in air_visit_data.columns if i in sample_submission.columns]
train = air_visit_data.copy()
test = sample_submission.copy()

In [ ]:
train_2017 = train.loc[(train.visit_date_year>=2017)].copy()

In [ ]:
# mean visitors by id year 2017
mean_v_id_2017 = train_2017[['visitors','air_store_id']].groupby('air_store_id').agg('mean').reset_index()
mean_v_id_2017.columns = ['air_store_id','mean_v_id_2017']

# mean visitors by month
mean_v_month = train[['visitors','visit_date_month']].groupby(['visit_date_month']).agg('mean').reset_index()
mean_v_month.columns = ['visit_date_month','mean_v_month']

# mean visitors by id & day_of_week 2017
mean_v_id_dow_2017 = train_2017[['visitors','air_store_id','visit_date_dayofw']].groupby(['air_store_id','visit_date_dayofw']).agg('mean').reset_index()
mean_v_id_dow_2017.columns = ['air_store_id','visit_date_dayofw','mean_v_id_dow_2017']

# mean visitors by day_of_week 2017
mean_v_dow_2017 = train_2017[['visitors','visit_date_dayofw']].groupby(['visit_date_dayofw']).agg('mean').reset_index()
mean_v_dow_2017.columns = ['visit_date_dayofw','mean_v_dow_2017']

# mean visitors by id
mean_v_id = train[['visitors','air_store_id']].groupby('air_store_id').agg('mean').reset_index()
mean_v_id.columns = ['air_store_id','mean_v_id']

# mean visitors by id & day_of_week
mean_v_id_dow = train[['visitors','air_store_id','visit_date_dayofw']].groupby(['air_store_id','visit_date_dayofw']).agg('mean').reset_index()
mean_v_id_dow.columns = ['air_store_id','visit_date_dayofw','mean_v_id_dow']

# mean visitors by day_of_week
mean_v_dow = train[['visitors','visit_date_dayofw']].groupby(['visit_date_dayofw']).agg('mean').reset_index()
mean_v_dow.columns = ['visit_date_dayofw','mean_v_dow']


# median visitors by id 2017
median_v_id_2017 = train_2017[['visitors','air_store_id']].groupby('air_store_id').agg('median').reset_index()
median_v_id_2017.columns = ['air_store_id','median_v_id_2017']

# median visitors by month
median_v_month = train[['visitors','visit_date_month']].groupby(['visit_date_month']).agg('median').reset_index()
median_v_month.columns = ['visit_date_month','median_v_month']

# median visitors by id and day_of_week 2017
median_v_id_dow_2017 = train_2017[['visitors','air_store_id','visit_date_dayofw']].groupby(['air_store_id','visit_date_dayofw']).agg('median').reset_index()
median_v_id_dow_2017.columns = ['air_store_id','visit_date_dayofw','median_v_id_dow_2017']

# median visitors by day_of_week 2017
median_v_dow_2017 = train_2017[['visitors','visit_date_dayofw']].groupby(['visit_date_dayofw']).agg('median').reset_index()
median_v_dow_2017.columns = ['visit_date_dayofw','median_v_dow_2017']

# median visitors by id
median_v_id = train[['visitors','air_store_id']].groupby('air_store_id').agg('median').reset_index()
median_v_id.columns = ['air_store_id','median_v_id']

# mean visitors by id & day_of_week
median_v_id_dow = train[['visitors','air_store_id','visit_date_dayofw']].groupby(['air_store_id','visit_date_dayofw']).agg('median').reset_index()
median_v_id_dow.columns = ['air_store_id','visit_date_dayofw','median_v_id_dow']

# mean visitors by day_of_week
median_v_dow = train[['visitors','visit_date_dayofw']].groupby(['visit_date_dayofw']).agg('median').reset_index()
median_v_dow.columns = ['visit_date_dayofw','median_v_dow']



In [ ]:
mean_v_id_2017.head(3)

,air_store_id,mean_v_id_2017
0,air_00a91d42b08b08d9,28.122222
1,air_0164b9927d20bcc3,9.048193
2,air_0241aa3964b7f861,8.373626


In [ ]:
mean_v_month.head(3)

,visit_date_month,mean_v_month
0,1,19.976758
1,2,20.477695
2,3,22.400360


In [ ]:
mean_v_id_dow_2017.head(3)

,air_store_id,visit_date_dayofw,mean_v_id_dow_2017
0,air_00a91d42b08b08d9,0,25.615385
1,air_00a91d42b08b08d9,1,30.533333
2,air_00a91d42b08b08d9,2,29.750000


In [ ]:
train = air_visit_data.copy()
test = sample_submission.copy()
train = train[columns]
test = test[columns]

# merging train and test
train = train.merge(mean_v_id_2017,on='air_store_id',how='left')
test = test.merge(mean_v_id_2017,on='air_store_id',how='left')
train = train.merge(mean_v_month,on=['visit_date_month'],how='left')
test = test.merge(mean_v_month,on= ['visit_date_month'],how='left')
train = train.merge(mean_v_id_dow_2017,on=['air_store_id','visit_date_dayofw'],how='left')
test = test.merge(mean_v_id_dow_2017,on= ['air_store_id','visit_date_dayofw'],how='left')
train = train.merge(mean_v_dow_2017,on=['visit_date_dayofw'],how='left')
test = test.merge(mean_v_dow_2017,on= ['visit_date_dayofw'],how='left')
train = train.merge(mean_v_id,on=['air_store_id'],how='left')
test = test.merge(mean_v_id,on= ['air_store_id'],how='left')
train = train.merge(mean_v_id_dow,on=['air_store_id','visit_date_dayofw'],how='left')
test = test.merge(mean_v_id_dow,on= ['air_store_id','visit_date_dayofw'],how='left')
train = train.merge(mean_v_dow,on=['visit_date_dayofw'],how='left')
test = test.merge(mean_v_dow,on= ['visit_date_dayofw'],how='left')

train = train.merge(median_v_id_2017,on='air_store_id',how='left')
test = test.merge(median_v_id_2017,on='air_store_id',how='left')
train = train.merge(median_v_month,on=['visit_date_month'],how='left')
test = test.merge(median_v_month,on= ['visit_date_month'],how='left')
train = train.merge(median_v_id_dow_2017,on=['air_store_id','visit_date_dayofw'],how='left')
test = test.merge(median_v_id_dow_2017,on= ['air_store_id','visit_date_dayofw'],how='left')
train = train.merge(median_v_dow_2017,on=['visit_date_dayofw'],how='left')
test = test.merge(median_v_dow_2017,on= ['visit_date_dayofw'],how='left')
train = train.merge(median_v_id,on=['air_store_id'],how='left')
test = test.merge(median_v_id,on= ['air_store_id'],how='left')
train = train.merge(median_v_id_dow,on=['air_store_id','visit_date_dayofw'],how='left')
test = test.merge(median_v_id_dow,on= ['air_store_id','visit_date_dayofw'],how='left')
train = train.merge(median_v_dow,on=['visit_date_dayofw'],how='left')
test = test.merge(median_v_dow,on= ['visit_date_dayofw'],how='left')

train = train.merge(df_open_restro,on=['visit_date_year','weekofyear'],how='left')
test = test.merge(df_open_restro,on= ['visit_date_year','weekofyear'],how='left')

# getting id's from test set
ids = sample_submission['id'].values

In [ ]:
train.head(3)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,2,2016,13,2,17.878788,19.976758,18.2000,18.967690,22.782609,23.843750,19.230121,17.0,16,19.0,15,22.0,25.0,16,297.0
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,3,2016,14,2,17.878788,19.976758,14.4375,18.199442,22.782609,20.292308,18.922702,17.0,16,14.5,15,22.0,21.0,16,297.0
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,4,2016,15,2,17.878788,19.976758,33.1875,23.080806,22.782609,34.738462,23.072737,17.0,16,33.0,19,22.0,35.0,19,297.0


In [ ]:
test.head(3)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr
0,air_00a91d42b08b08d9,2017-04-23,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0,4,6,2017,23,16,28.122222,21.816002,NaN,23.912740,26.081897,2.000000,23.873362,28.0,18,NaN,19,26.0,2.0,19,822
1,air_00a91d42b08b08d9,2017-04-24,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,0,2017,24,17,28.122222,21.816002,25.615385,17.146256,26.081897,22.457143,17.177009,28.0,18,20.0,13,26.0,19.0,14,821
2,air_00a91d42b08b08d9,2017-04-25,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,1,2017,25,17,28.122222,21.816002,30.533333,17.549730,26.081897,24.350000,17.672137,28.0,18,30.0,14,26.0,24.5,14,821


## 10.Processing weekend and holidays data

In [ ]:
date_info['calendar_date']  = pd.to_datetime(date_info['calendar_date'],format= '%Y-%m-%d')
date_info.rename(columns = {'calendar_date':'visit_date'},inplace = True)
# get weekend and holidays
wkend_holidays = date_info.apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
# add decreasing weights from now
date_info['weight'] = ((date_info.index + 1.0) / len(date_info)) ** 5.0
# merge tran & test
train = train.merge(date_info,on='visit_date',how='left')
test = test.merge(date_info,on='visit_date',how='left')

In [ ]:
train.head(3)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,2,2016,13,2,17.878788,19.976758,18.2000,18.967690,22.782609,23.843750,19.230121,17.0,16,19.0,15,22.0,25.0,16,297.0,Wednesday,0,1.005227e-08
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,3,2016,14,2,17.878788,19.976758,14.4375,18.199442,22.782609,20.292308,18.922702,17.0,16,14.5,15,22.0,21.0,16,297.0,Thursday,0,1.456087e-08
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,4,2016,15,2,17.878788,19.976758,33.1875,23.080806,22.782609,34.738462,23.072737,17.0,16,33.0,19,22.0,35.0,19,297.0,Friday,0,2.055908e-08


In [ ]:
test.head(3)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight
0,air_00a91d42b08b08d9,2017-04-23,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0,4,6,2017,23,16,28.122222,21.816002,NaN,23.912740,26.081897,2.000000,23.873362,28.0,18,NaN,19,26.0,2.0,19,822,Sunday,0,0.682692
1,air_00a91d42b08b08d9,2017-04-24,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,0,2017,24,17,28.122222,21.816002,25.615385,17.146256,26.081897,22.457143,17.177009,28.0,18,20.0,13,26.0,19.0,14,821,Monday,0,0.689848
2,air_00a91d42b08b08d9,2017-04-25,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,1,2017,25,17,28.122222,21.816002,30.533333,17.549730,26.081897,24.350000,17.672137,28.0,18,30.0,14,26.0,24.5,14,821,Tuesday,0,0.697064


In [ ]:
relation = pd.read_csv(root_path +'store_id_relation.csv')
relation['both'] = 1

relation.head(3)

,air_store_id,hpg_store_id,both
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a,1
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809,1
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9,1


In [ ]:
# merge with tra & test
train = train.merge(relation,how='left',on='air_store_id')
test = test.merge(relation,how='left',on='air_store_id')

In [ ]:

train = train.merge(hpg_store_info[['hpg_store_id','hpg_genre_name','hpg_area_name']],how='left',on='hpg_store_id')
test = test.merge(hpg_store_info[['hpg_store_id','hpg_genre_name','hpg_area_name']],how='left',on='hpg_store_id')

In [ ]:
hpg_store_info

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
...,...,...,...,...,...
4685,hpg_c6ed353a48c7c93e,Sichuan food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4686,hpg_a58d2423360b2fb4,Sichuan food,Tōkyō-to Minato-ku Roppongi,35.666137,139.733978
4687,hpg_d2021704076cdd70,Shanghai food,Tōkyō-to Chiyoda-ku None,35.695780,139.768453
4688,hpg_602472570d65e219,Spain/Mediterranean cuisine,Tōkyō-to Toshima-ku None,35.730068,139.711742


In [ ]:
train.head(3)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight,hpg_store_id,both,hpg_genre_name,hpg_area_name
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,2,2016,13,2,17.878788,19.976758,18.2000,18.967690,22.782609,23.843750,19.230121,17.0,16,19.0,15,22.0,25.0,16,297.0,Wednesday,0,1.005227e-08,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,3,2016,14,2,17.878788,19.976758,14.4375,18.199442,22.782609,20.292308,18.922702,17.0,16,14.5,15,22.0,21.0,16,297.0,Thursday,0,1.456087e-08,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,4,2016,15,2,17.878788,19.976758,33.1875,23.080806,22.782609,34.738462,23.072737,17.0,16,33.0,19,22.0,35.0,19,297.0,Friday,0,2.055908e-08,NaN,NaN,NaN,NaN


## 11. Monthly visitors mean

In [ ]:
# calculating monthly visitors mean for each restaurants
month_wise_mean=train.groupby(['air_store_id','visit_date_month'],as_index=False)['visitors'].mean()
month_wise_mean = month_wise_mean.pivot(index='air_store_id', columns='visit_date_month', values='visitors').reset_index()
month_wise_mean.columns = ['air_store_id','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']

In [ ]:
month_wise_mean.head(3)

,air_store_id,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,air_00a91d42b08b08d9,25.869565,29.652174,29.384615,27.222222,NaN,NaN,26.080000,24.166667,22.833333,23.250000,22.62500,30.428571
1,air_0164b9927d20bcc3,10.789474,7.476190,9.625000,8.315789,NaN,NaN,NaN,NaN,NaN,10.000000,8.26087,10.333333
2,air_0241aa3964b7f861,9.540000,9.212766,9.962264,10.250000,10.481481,10.24,8.178571,10.125000,10.954545,9.153846,9.60000,12.000000


In [ ]:
# merging with train & test data 
train = train.merge(month_wise_mean,on='air_store_id',how='left')
test = test.merge(month_wise_mean,on='air_store_id',how='left')

## 14.Weekday-wise visitors mean

In [ ]:
# calculating weekly visitors mean for each restaurants
weekday_wise_mean=train.groupby(['air_store_id','visit_date_dayofw'],as_index=False)['visitors'].mean()
weekday_wise_mean = weekday_wise_mean.pivot(index='air_store_id', columns='visit_date_dayofw', values='visitors').reset_index()
weekday_wise_mean.columns = ['air_store_id','w0','w1','w2','w3','w4','w5','w6']

# merging with train & test data 
train = train.merge(weekday_wise_mean,on='air_store_id',how='left')
test = test.merge(weekday_wise_mean,on='air_store_id',how='left')

In [ ]:
weekday_wise_mean.head(3)

,air_store_id,w0,w1,w2,w3,w4,w5,w6
0,air_00a91d42b08b08d9,22.457143,24.350000,28.125000,29.868421,36.500000,14.973684,2.00000
1,air_0164b9927d20bcc3,7.500000,9.560000,9.678571,9.846154,11.464286,6.409091,NaN
2,air_0241aa3964b7f861,8.920635,8.621212,9.852941,5.555556,7.786885,13.892308,10.78125


## 15.Add weather, (including 30 60 mean)

In [ ]:
train.shape, test.shape

((252108, 75), (32019, 75))

In [ ]:
type(pd.to_datetime(all_station_weather["visit_date"][0]))

pandas.core.series.Series

In [1]:
share_weather_path = "Restaurant_Forecast_6420/"

feature = pd.read_csv( share_weather_path + 'feature_manifest.csv').rename(columns={'id':'station_id'})
hpg_w = pd.read_csv( share_weather_path + 'hpg_store_info_with_nearest_active_station.csv')
air_w = pd.read_csv( share_weather_path +'air_store_info_with_nearest_active_station.csv')

hpg_w = hpg_w.drop(columns = hpg_w.columns[8:])
air_w = air_w.drop(columns = air_w.columns[8:])

NameError: ignored

In [ ]:
# import weather data for specific stations
hpg_station_list = hpg_w.station_id.unique()
air_station_list = air_w.station_id.unique()
all_station_arrray = np.unique(np.concatenate((hpg_station_list,air_station_list),axis=None))

weather_feature = ['station_id','calendar_date','avg_temperature','hours_sunlight','cloud_cover']

def import_weather (station_list):
  result = []
  for station in station_list:
    temp = pd.read_csv( share_weather_path + 'Weather/station_weather/'+ station +'.csv')
    temp['station_id'] = station
    temp = temp[weather_feature]
    result.append(temp)
  
  station_weather = pd.concat(result)
  return station_weather


all_station_weather = import_weather(all_station_arrray)
all_station_weather = all_station_weather.rename(columns={"calendar_date":"visit_date"})
air_wN = air_w[["air_store_id","air_area_name","station_id"]]

train = train.merge(air_wN, how = "left", on = ["air_store_id", "air_area_name"])
test = test.merge(air_wN, how = "left", on = ["air_store_id", "air_area_name"])

all_station_weather.visit_date = all_station_weather.visit_date.astype('datetime64[ns]')

In [ ]:
all_station_weather.head(2)

,station_id,visit_date,avg_temperature,hours_sunlight,cloud_cover
0,fukuoka__dazaifu-kana__dasaif,2016-01-01,5.7,5.1,NaN
1,fukuoka__dazaifu-kana__dasaif,2016-01-02,10.2,1.3,NaN


In [ ]:
########### Be Careful #########


### This part of the code is adding weather 30/60 mean groupby rolling ###
### The outcome is acutally not as good as simply adding daily weather info ###
### Skip this part get "0.47746" on public score ###
### Run this part get "0.47897" on public socre ### 

w = all_station_weather.groupby('station_id')

# add column
all_station_weather['sum_31T'] = w['avg_temperature'].rolling(min_periods=1, window=31).sum().\
    to_frame().reset_index()['avg_temperature']
all_station_weather['sum_61T'] = w['avg_temperature'].rolling(min_periods=1, window=61).sum().\
    to_frame().reset_index()['avg_temperature']

all_station_weather['sum_31S'] = w['hours_sunlight'].rolling(min_periods=1, window=31).sum().\
    to_frame().reset_index()['hours_sunlight']
all_station_weather['sum_61S'] = w['hours_sunlight'].rolling(min_periods=1, window=61).sum().\
    to_frame().reset_index()['hours_sunlight']

all_station_weather['cnt_31W'] = w['avg_temperature'].rolling(min_periods=1, window=31).count().\
    to_frame().reset_index()['avg_temperature']
all_station_weather['cnt_61W'] = w['avg_temperature'].rolling(min_periods=1, window=61).count().\
    to_frame().reset_index()['avg_temperature']

all_station_weather['mean_30T']=(all_station_weather['sum_31T']-all_station_weather['avg_temperature'])/
(all_station_weather['cnt_31W']-1)
all_station_weather['mean_60T']=(all_station_weather['sum_61T']-all_station_weather['avg_temperature'])/
(all_station_weather['cnt_61W']-1)

all_station_weather['mean_30S']=(all_station_weather['sum_31S']-all_station_weather['hours_sunlight'])/
(all_station_weather['cnt_31W']-1)
all_station_weather['mean_60S']=(all_station_weather['sum_61S']-all_station_weather['hours_sunlight'])/
(all_station_weather['cnt_61W']-1)

all_station_weather = all_station_weather.fillna(0)
all_station_weather = all_station_weather.drop(columns=['sum_31T','sum_61T','sum_31S','sum_61S','cnt_31W','cnt_61W'])

all_station_weather

,station_id,visit_date,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S
0,fukuoka__dazaifu-kana__dasaif,2016-01-01,5.7,5.1,0.0,0.000000,0.000000,0.000000,0.000000
1,fukuoka__dazaifu-kana__dasaif,2016-01-02,10.2,1.3,0.0,5.700000,5.700000,5.100000,5.100000
2,fukuoka__dazaifu-kana__dasaif,2016-01-03,11.2,0.5,0.0,7.950000,7.950000,3.200000,3.200000
3,fukuoka__dazaifu-kana__dasaif,2016-01-04,9.4,4.2,0.0,9.033333,9.033333,2.300000,2.300000
4,fukuoka__dazaifu-kana__dasaif,2016-01-05,8.8,0.0,0.0,9.125000,9.125000,2.775000,2.775000
...,...,...,...,...,...,...,...,...,...
512,yamaguchi__shimonoseki-kana__shimonoseki,2017-05-27,19.2,13.5,0.0,19.553333,17.243333,7.470000,6.511667
513,yamaguchi__shimonoseki-kana__shimonoseki,2017-05-28,21.3,12.6,0.0,19.673333,17.405000,7.706667,6.560000
514,yamaguchi__shimonoseki-kana__shimonoseki,2017-05-29,23.2,12.8,0.0,19.890000,17.596667,7.713333,6.766667
515,yamaguchi__shimonoseki-kana__shimonoseki,2017-05-30,23.6,12.1,6.3,20.063333,17.775000,7.730000,6.885000


In [ ]:
train = train.merge(all_station_weather, how = "left", on = ["station_id", "visit_date"])
test = test.merge(all_station_weather, how = "left", on = ["station_id", "visit_date"])


In [ ]:
train.head(2)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight,hpg_store_id,both,hpg_genre_name,hpg_area_name,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,w0,w1,w2,w3,w4,w5,w6,station_id,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,2,2016,13,2,17.878788,19.976758,18.2000,18.967690,22.782609,23.843750,19.230121,17.0,16,19.0,15,22.0,25.0,16,297.0,Wednesday,0,1.005227e-08,NaN,NaN,NaN,NaN,18.35,19.77551,20.618182,21.152174,28.478261,30.807692,28.16,24.782609,26.64,23.346154,19.111111,22.269231,13.754386,18.580645,23.84375,20.292308,34.738462,27.651515,3.666667,tokyo__tokyo-kana__tonokyo,4.3,8.7,2.5,7.625,7.625,0.866667,0.866667
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,3,2016,14,2,17.878788,19.976758,14.4375,18.199442,22.782609,20.292308,18.922702,17.0,16,14.5,15,22.0,21.0,16,297.0,Thursday,0,1.456087e-08,NaN,NaN,NaN,NaN,18.35,19.77551,20.618182,21.152174,28.478261,30.807692,28.16,24.782609,26.64,23.346154,19.111111,22.269231,13.754386,18.580645,23.84375,20.292308,34.738462,27.651515,3.666667,tokyo__tokyo-kana__tonokyo,6.0,9.1,0.5,7.300,7.300,0.784615,0.784615


In [ ]:
test.head(2)

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight,hpg_store_id,both,hpg_genre_name,hpg_area_name,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,w0,w1,w2,w3,w4,w5,w6,station_id,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S
0,air_00a91d42b08b08d9,2017-04-23,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0,4,6,2017,23,16,28.122222,21.816002,NaN,23.912740,26.081897,2.000000,23.873362,28.0,18,NaN,19,26.0,2.0,19,822,Sunday,0,0.682692,hpg_fbe603376b5980fc,1.0,NaN,NaN,25.869565,29.652174,29.384615,27.222222,NaN,NaN,26.08,24.166667,22.833333,23.25,22.625,30.428571,22.457143,24.35,28.125,29.868421,36.5,14.973684,2.0,tokyo__tokyo-kana__tonokyo,13.6,10.5,4.3,13.973333,11.186667,5.076667,5.266667
1,air_00a91d42b08b08d9,2017-04-24,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,0,2017,24,17,28.122222,21.816002,25.615385,17.146256,26.081897,22.457143,17.177009,28.0,18,20.0,13,26.0,19.0,14,821,Monday,0,0.689848,hpg_fbe603376b5980fc,1.0,NaN,NaN,25.869565,29.652174,29.384615,27.222222,NaN,NaN,26.08,24.166667,22.833333,23.25,22.625,30.428571,22.457143,24.35,28.125,29.868421,36.5,14.973684,2.0,tokyo__tokyo-kana__tonokyo,14.5,7.0,7.0,14.203333,11.325000,5.480000,5.525000


## 16.Last 30/60 days visitors

In [ ]:
share_path = "drive/Shareddrives/Restaurant_Forecast_6420/"
air_visit_all_date = pd.read_csv( share_path + 'air_visit_all_date.csv')
air_visit_all_date = air_visit_all_date.drop(columns=['Unnamed: 0'])
air_visit_all_date['visitors'] = air_visit_all_date.visitors.astype('int64')
air_visit_all_date['date'] =  pd.to_datetime(air_visit_all_date['date'])
air_visit_all_date = air_visit_all_date.rename(columns = {'date': 'visit_date'})

air_visit_all_date = pd.concat([air_visit_all_date,sample_submission[['visit_date','air_store_id','visitors']]])
air_visit_all_date = air_visit_all_date.sort_values(by=['air_store_id','visit_date']).reset_index(drop=True)

air_visit_all_date

# air_visit_all_date.dtypes

,visit_date,air_store_id,visitors
0,2016-07-01,air_00a91d42b08b08d9,35
1,2016-07-02,air_00a91d42b08b08d9,9
2,2016-07-03,air_00a91d42b08b08d9,0
3,2016-07-04,air_00a91d42b08b08d9,20
4,2016-07-05,air_00a91d42b08b08d9,25
...,...,...,...
328293,2017-05-27,air_fff68b929994bfbd,0
328294,2017-05-28,air_fff68b929994bfbd,0
328295,2017-05-29,air_fff68b929994bfbd,0
328296,2017-05-30,air_fff68b929994bfbd,0


In [ ]:
# groupby rolling
x = air_visit_all_date.groupby('air_store_id')

# add column
air_visit_all_date['sum_31'] = x['visitors'].rolling(min_periods=1, window=31).sum().\
    to_frame().reset_index()['visitors']
air_visit_all_date['sum_61'] = x['visitors'].rolling(min_periods=1, window=61).sum().\
    to_frame().reset_index()['visitors']

air_visit_all_date['cnt_31'] = x['visitors'].rolling(min_periods=1, window=31).count().\
    to_frame().reset_index()['visitors']
air_visit_all_date['cnt_61'] = x['visitors'].rolling(min_periods=1, window=61).count().\
    to_frame().reset_index()['visitors']

# mean
air_visit_all_date['mean_30']=(air_visit_all_date['sum_31']-air_visit_all_date['visitors'])/(air_visit_all_date['cnt_31']-1)
air_visit_all_date['mean_60']=(air_visit_all_date['sum_61']-air_visit_all_date['visitors'])/(air_visit_all_date['cnt_61']-1)
air_visit_all_date = air_visit_all_date.fillna(0)
air_visit_all_date = air_visit_all_date.drop(columns=['visitors','sum_31','sum_61','cnt_31','cnt_61'])

air_visit_all_date

,visit_date,air_store_id,mean_30,mean_60
0,2016-07-01,air_00a91d42b08b08d9,0.000000,0.000000
1,2016-07-02,air_00a91d42b08b08d9,35.000000,35.000000
2,2016-07-03,air_00a91d42b08b08d9,22.000000,22.000000
3,2016-07-04,air_00a91d42b08b08d9,14.666667,14.666667
4,2016-07-05,air_00a91d42b08b08d9,16.000000,16.000000
...,...,...,...,...
328293,2017-05-27,air_fff68b929994bfbd,0.000000,2.050000
328294,2017-05-28,air_fff68b929994bfbd,0.000000,1.900000
328295,2017-05-29,air_fff68b929994bfbd,0.000000,1.866667
328296,2017-05-30,air_fff68b929994bfbd,0.000000,1.783333


In [ ]:
# merging with train & test data 
train = train.merge(air_visit_all_date, on=['air_store_id','visit_date'],how='left')
test = test.merge(air_visit_all_date, on=['air_store_id','visit_date'],how='left')

In [ ]:
print(train.shape)
train.head(3)

(252108, 77)


,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight,hpg_store_id,both,hpg_genre_name,hpg_area_name,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,w0,w1,w2,w3,w4,w5,w6,station_id,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S,mean_30,mean_60
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,2,2016,13,2,17.878788,19.976758,18.2000,18.967690,22.782609,23.843750,19.230121,17.0,16,19.0,15,22.0,25.0,16,297.0,Wednesday,0,1.005227e-08,NaN,NaN,NaN,NaN,18.35,19.77551,20.618182,21.152174,28.478261,30.807692,28.16,24.782609,26.64,23.346154,19.111111,22.269231,13.754386,18.580645,23.84375,20.292308,34.738462,27.651515,3.666667,tokyo__tokyo-kana__tonokyo,4.3,8.7,2.5,7.625000,7.625000,0.866667,0.866667,0.0,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,3,2016,14,2,17.878788,19.976758,14.4375,18.199442,22.782609,20.292308,18.922702,17.0,16,14.5,15,22.0,21.0,16,297.0,Thursday,0,1.456087e-08,NaN,NaN,NaN,NaN,18.35,19.77551,20.618182,21.152174,28.478261,30.807692,28.16,24.782609,26.64,23.346154,19.111111,22.269231,13.754386,18.580645,23.84375,20.292308,34.738462,27.651515,3.666667,tokyo__tokyo-kana__tonokyo,6.0,9.1,0.5,7.300000,7.300000,0.784615,0.784615,25.0,25.0
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,0,8,51,0.0,0.0,0.0,0.0,0.0,0.0,22.663399,21.5,61,2,1,4,2016,15,2,17.878788,19.976758,33.1875,23.080806,22.782609,34.738462,23.072737,17.0,16,33.0,19,22.0,35.0,19,297.0,Friday,0,2.055908e-08,NaN,NaN,NaN,NaN,18.35,19.77551,20.618182,21.152174,28.478261,30.807692,28.16,24.782609,26.64,23.346154,19.111111,22.269231,13.754386,18.580645,23.84375,20.292308,34.738462,27.651515,3.666667,tokyo__tokyo-kana__tonokyo,5.6,8.1,6.0,7.078571,7.078571,1.442857,1.442857,28.5,28.5


In [ ]:
print(test.shape)
test.head(2)

(32019, 77)


,air_store_id,visit_date,visitors,air_genre_name,air_area_name,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,day_of_week,holiday_flg,weight,hpg_store_id,both,hpg_genre_name,hpg_area_name,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,w0,w1,w2,w3,w4,w5,w6,station_id,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S,mean_30,mean_60
0,air_00a91d42b08b08d9,2017-04-23,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1,4,20,0.0,0.0,0.0,0.0,0.0,0.0,14.800000,11.0,99.0,2.0,4,6,2017,23,16,28.122222,21.816002,NaN,23.912740,26.081897,2.000000,23.873362,28.0,18,NaN,19,26.0,2.0,19,822,Sunday,0,0.682692,hpg_fbe603376b5980fc,1.0,NaN,NaN,25.869565,29.652174,29.384615,27.222222,NaN,NaN,26.08,24.166667,22.833333,23.25,22.625,30.428571,22.457143,24.35,28.125,29.868421,36.5,14.973684,2.0,tokyo__tokyo-kana__tonokyo,13.6,10.5,4.3,13.973333,11.186667,5.076667,5.266667,24.233333,23.65
1,air_00a91d42b08b08d9,2017-04-24,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,0,4,20,0.0,0.0,0.0,0.0,0.0,0.0,28.432292,28.0,57.0,1.0,4,0,2017,24,17,28.122222,21.816002,25.615385,17.146256,26.081897,22.457143,17.177009,28.0,18,20.0,13,26.0,19.0,14,821,Monday,0,0.689848,hpg_fbe603376b5980fc,1.0,NaN,NaN,25.869565,29.652174,29.384615,27.222222,NaN,NaN,26.08,24.166667,22.833333,23.25,22.625,30.428571,22.457143,24.35,28.125,29.868421,36.5,14.973684,2.0,tokyo__tokyo-kana__tonokyo,14.5,7.0,7.0,14.203333,11.325000,5.480000,5.525000,23.100000,23.10


In [ ]:
train = train.fillna(0) 
test = test.fillna(0)

In [ ]:
# dropping unimportant columns
train_fe = train.drop(['hpg_area_name','hpg_genre_name','day_of_week'],axis =1)
test_fe = test.drop(['hpg_area_name','hpg_genre_name','day_of_week'],axis =1)

In [ ]:
train_fe.to_csv(root_path +'train_30VW.csv')
test_fe.to_csv(root_path +'test_30VW.csv')

# Modeling 

In [ ]:
train.columns

Index(['air_store_id', 'visit_date', 'visitors', 'air_genre_name',
       'air_area_name', 'latitude', 'longitude', 'holiday_eve', 'non_working',
       'genre_in_area', 'total_r_in_area', 'reserve_visitors', 'reserve_-12_h',
       'reserve_12_36_h', 'reserve_36_60_h', 'reserve_60_84_h',
       'reserve_84+_h', 'visitors_mean', 'visitors_median', 'visitors_max',
       'visitors_min', 'visit_date_month', 'visit_date_dayofw',
       'visit_date_year', 'visit_date_dayofm', 'weekofyear', 'mean_v_id_2017',
       'mean_v_month', 'mean_v_id_dow_2017', 'mean_v_dow_2017', 'mean_v_id',
       'mean_v_id_dow', 'mean_v_dow', 'median_v_id_2017', 'median_v_month',
       'median_v_id_dow_2017', 'median_v_dow_2017', 'median_v_id',
       'median_v_id_dow', 'median_v_dow', 'no_open_restr', 'day_of_week',
       'holiday_flg', 'weight', 'hpg_store_id', 'both', 'hpg_genre_name',
       'hpg_area_name', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9',
       'm10', 'm11', 'm12', 'w0', 'w1', 'w2',

In [ ]:
train_VW = train.drop(columns=["hpg_store_id","station_id", "both", 'hpg_genre_name', 'hpg_area_name'])
test_VW = test.drop(columns=["hpg_store_id","station_id", "both", 'hpg_genre_name', 'hpg_area_name'])

# Encode labels of categorical columns:
cat_features = [col for col in ['air_genre_name', 'air_area_name', 'day_of_week']]
for column in cat_features:
    temp_prep = pd.get_dummies(pd.Series(train_VW[column]))
    train_VW = pd.concat([train_VW,temp_prep],axis=1)
    train_VW = train_VW.drop([column],axis=1)
    temp_predict = pd.get_dummies(pd.Series(test_VW[column]))
    test_VW = pd.concat([test_VW,temp_predict],axis=1)
    test_VW = test_VW.drop([column],axis=1)
    for missing_col in temp_prep:     # Make sure the columns of train and test are identical
        if missing_col not in test_VW.columns:
            test_VW[missing_col] = 0
    for missing_col in temp_predict:     # Make sure the columns of train and test are identical
        if missing_col not in train_VW.columns:
            train_VW[missing_col] = 0  


train_VW['visitors'] = np.log1p(train_VW['visitors'])

# Encode 
###J_train.drop(['visit_date'], axis=1, inplace=True)
label_enc = preprocessing.LabelEncoder()
label_enc.fit(train_VW['air_store_id'])
train_VW['air_store_id'] = label_enc.transform(train_VW['air_store_id'])
prep_cols = train_VW.columns

###J_test.drop(['visit_date'], axis=1, inplace=True)  
test_VW['air_store_id'] = label_enc.transform(test_VW['air_store_id'])

train_VW = train_VW.drop(columns=["visit_date"])

#X_train = X_train.drop(columns=["visit_date"])
#X_test = X_test.drop(columns=["visit_date"])

In [ ]:
test_VW

,air_store_id,visit_date,visitors,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,holiday_flg,weight,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,w0,w1,w2,w3,w4,w5,w6,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S,mean_30,mean_60,Asian,Bar/Cocktail,Cafe/Sweets,Creative cuisine,Dining bar,International cuisine,Italian/French,Izakaya,Japanese food,Karaoke/Party,Okonomiyaki/Monja/Teppanyaki,Other,Western food,Yakiniku/Korean food,Fukuoka-ken Fukuoka-shi Daimyō,Fukuoka-ken Fukuoka-shi Hakata Ekimae,Fukuoka-ken Fukuoka-shi Imaizumi,Fukuoka-ken Fukuoka-shi Momochi,Fukuoka-ken Fukuoka-shi Shiobaru,Fukuoka-ken Fukuoka-shi Takatori,Fukuoka-ken Fukuoka-shi Tenjin,Fukuoka-ken Fukuoka-shi Torikai,Fukuoka-ken Itoshima-shi Maebarunishi,Fukuoka-ken Kitakyūshū-shi Konyamachi,Fukuoka-ken Kitakyūshū-shi None,Fukuoka-ken Kitakyūshū-shi Ōtemachi,Fukuoka-ken Kurume-shi Jōnanmachi,Fukuoka-ken Yame-shi Motomachi,Hiroshima-ken Fukuyama-shi Higashisakuramachi,Hiroshima-ken Hiroshima-shi Kokutaijimachi,Hiroshima-ken Hiroshima-shi Nagarekawachō,Hokkaidō Abashiri-shi Minami 6 Jōhigashi,Hokkaidō Asahikawa-shi 6 Jōdōri,Hokkaidō Katō-gun Motomachi,Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,Hokkaidō Sapporo-shi Kita 24 Jōnishi,Hokkaidō Sapporo-shi Kotoni 2 Jō,Hokkaidō Sapporo-shi Minami 3 Jōnishi,Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,Hyōgo-ken Himeji-shi Yasuda,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,Hyōgo-ken Kōbe-shi Kumoidōri,Hyōgo-ken Kōbe-shi Motomachidōri,Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,Hyōgo-ken Nishinomiya-shi Rokutanjichō,Hyōgo-ken Takarazuka-shi Tōyōchō,Miyagi-ken Sendai-shi Kamisugi,Niigata-ken Kashiwazaki-shi Chūōchō,Niigata-ken Nagaoka-shi Ōtedōri,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,Niigata-ken Niigata-shi Teraohigashi,Shizuoka-ken Hamamatsu-shi Motoshirochō,Shizuoka-ken Numazu-shi Miyukichō,Shizuoka-ken Shizuoka-shi Ōtemachi,Tōkyō-to Adachi-ku Chūōhonchō,Tōkyō-to Bunkyō-ku Kasuga,Tōkyō-to Bunkyō-ku Sendagi,Tōkyō-to Chiyoda-ku Kanda Jinbōchō,Tōkyō-to Chiyoda-ku Kudanminami,Tōkyō-to Chūō-ku Ginza,Tōkyō-to Chūō-ku Tsukiji,Tōkyō-to Edogawa-ku Chūō,Tōkyō-to Fuchū-shi Miyanishichō,Tōkyō-to Itabashi-ku Itabashi,Tōkyō-to Katsushika-ku Tateishi,Tōkyō-to Kita-ku Ōjihonchō,Tōkyō-to Koganei-shi Honchō,Tōkyō-to Kōtō-ku Kameido,Tōkyō-to Kōtō-ku Tomioka,Tōkyō-to Kōtō-ku Tōyō,Tōkyō-to Machida-shi Morino,Tōkyō-to Meguro-ku Jiyūgaoka,Tōkyō-to Meguro-ku Kamimeguro,Tōkyō-to Meguro-ku Takaban,Tōkyō-to Minato-ku Akasaka,Tōkyō-to Minato-ku Roppongi,Tōkyō-to Minato-ku Shibakōen,Tōkyō-to Musashino-shi Kichijōji Honchō,Tōkyō-to Musashino-shi Midorichō,Tōkyō-to Nakano-ku Nakano,Tōkyō-to Nerima-ku Toyotamakita,Tōkyō-to Setagaya-ku Kitazawa,Tōkyō-to Setagaya-ku Okusawa,Tōkyō-to Setagaya-ku Sangenjaya,Tōkyō-to Setagaya-ku Setagaya,Tōkyō-to Shibuya-ku Dōgenzaka,Tōkyō-to Shibuya-ku Higashi,Tōkyō-to Shibuya-ku Jingūmae,Tōkyō-to Shibuya-ku Shibuya,Tōkyō-to Shinagawa-ku Higashigotanda,Tōkyō-to Shinagawa-ku Hiromachi,Tōkyō-to Shinjuku-ku Kabukichō,Tōkyō-to Shinjuku-ku Nishishinjuku,Tōkyō-to Suginami-ku Asagayaminami,Tōkyō-to Tachikawa-shi Izumichō,Tōkyō-to Taitō-ku Asakusa,Tōkyō-to Taitō-ku Higashiueno,Tōkyō-to Taitō-ku Kuramae,Tōkyō-to Toshima-ku Higashiikebukuro,Tōkyō-to Toshima-ku Mejiro,Tōkyō-to Toshima-ku Minamiikebukuro,Tōkyō-to Toshima-ku Nishiikebukuro,Tōkyō-to Toshima-ku Sugamo,Tōkyō-to Ōta-ku Kamata,Tōkyō-to Ōta-ku Kamiikedai,Ōsaka-fu Higashiōsaka-shi Aramotokita,Ōsaka-fu Neyagawa-shi Honmachi,Ōsaka-fu Sakai-shi Minamikawaramachi,Ōsaka-fu Suita-shi

In [ ]:
train_VW

,air_store_id,visitors,latitude,longitude,holiday_eve,non_working,genre_in_area,total_r_in_area,reserve_visitors,reserve_-12_h,reserve_12_36_h,reserve_36_60_h,reserve_60_84_h,reserve_84+_h,visitors_mean,visitors_median,visitors_max,visitors_min,visit_date_month,visit_date_dayofw,visit_date_year,visit_date_dayofm,weekofyear,mean_v_id_2017,mean_v_month,mean_v_id_dow_2017,mean_v_dow_2017,mean_v_id,mean_v_id_dow,mean_v_dow,median_v_id_2017,median_v_month,median_v_id_dow_2017,median_v_dow_2017,median_v_id,median_v_id_dow,median_v_dow,no_open_restr,holiday_flg,weight,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,w0,w1,w2,w3,w4,w5,w6,avg_temperature,hours_sunlight,cloud_cover,mean_30T,mean_60T,mean_30S,mean_60S,mean_30,mean_60,Asian,Bar/Cocktail,Cafe/Sweets,Creative cuisine,Dining bar,International cuisine,Italian/French,Izakaya,Japanese food,Karaoke/Party,Okonomiyaki/Monja/Teppanyaki,Other,Western food,Yakiniku/Korean food,Fukuoka-ken Fukuoka-shi Daimyō,Fukuoka-ken Fukuoka-shi Hakata Ekimae,Fukuoka-ken Fukuoka-shi Imaizumi,Fukuoka-ken Fukuoka-shi Momochi,Fukuoka-ken Fukuoka-shi Shiobaru,Fukuoka-ken Fukuoka-shi Takatori,Fukuoka-ken Fukuoka-shi Tenjin,Fukuoka-ken Fukuoka-shi Torikai,Fukuoka-ken Itoshima-shi Maebarunishi,Fukuoka-ken Kitakyūshū-shi Konyamachi,Fukuoka-ken Kitakyūshū-shi None,Fukuoka-ken Kitakyūshū-shi Ōtemachi,Fukuoka-ken Kurume-shi Jōnanmachi,Fukuoka-ken Yame-shi Motomachi,Hiroshima-ken Fukuyama-shi Higashisakuramachi,Hiroshima-ken Hiroshima-shi Kokutaijimachi,Hiroshima-ken Hiroshima-shi Nagarekawachō,Hokkaidō Abashiri-shi Minami 6 Jōhigashi,Hokkaidō Asahikawa-shi 6 Jōdōri,Hokkaidō Katō-gun Motomachi,Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,Hokkaidō Sapporo-shi Kita 24 Jōnishi,Hokkaidō Sapporo-shi Kotoni 2 Jō,Hokkaidō Sapporo-shi Minami 3 Jōnishi,Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,Hyōgo-ken Himeji-shi Yasuda,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,Hyōgo-ken Kōbe-shi Kumoidōri,Hyōgo-ken Kōbe-shi Motomachidōri,Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,Hyōgo-ken Nishinomiya-shi Rokutanjichō,Hyōgo-ken Takarazuka-shi Tōyōchō,Miyagi-ken Sendai-shi Kamisugi,Niigata-ken Kashiwazaki-shi Chūōchō,Niigata-ken Nagaoka-shi Ōtedōri,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,Niigata-ken Niigata-shi Teraohigashi,Shizuoka-ken Hamamatsu-shi Motoshirochō,Shizuoka-ken Numazu-shi Miyukichō,Shizuoka-ken Shizuoka-shi Ōtemachi,Tōkyō-to Adachi-ku Chūōhonchō,Tōkyō-to Bunkyō-ku Kasuga,Tōkyō-to Bunkyō-ku Sendagi,Tōkyō-to Chiyoda-ku Kanda Jinbōchō,Tōkyō-to Chiyoda-ku Kudanminami,Tōkyō-to Chūō-ku Ginza,Tōkyō-to Chūō-ku Tsukiji,Tōkyō-to Edogawa-ku Chūō,Tōkyō-to Fuchū-shi Miyanishichō,Tōkyō-to Itabashi-ku Itabashi,Tōkyō-to Katsushika-ku Tateishi,Tōkyō-to Kita-ku Ōjihonchō,Tōkyō-to Koganei-shi Honchō,Tōkyō-to Kōtō-ku Kameido,Tōkyō-to Kōtō-ku Tomioka,Tōkyō-to Kōtō-ku Tōyō,Tōkyō-to Machida-shi Morino,Tōkyō-to Meguro-ku Jiyūgaoka,Tōkyō-to Meguro-ku Kamimeguro,Tōkyō-to Meguro-ku Takaban,Tōkyō-to Minato-ku Akasaka,Tōkyō-to Minato-ku Roppongi,Tōkyō-to Minato-ku Shibakōen,Tōkyō-to Musashino-shi Kichijōji Honchō,Tōkyō-to Musashino-shi Midorichō,Tōkyō-to Nakano-ku Nakano,Tōkyō-to Nerima-ku Toyotamakita,Tōkyō-to Setagaya-ku Kitazawa,Tōkyō-to Setagaya-ku Okusawa,Tōkyō-to Setagaya-ku Sangenjaya,Tōkyō-to Setagaya-ku Setagaya,Tōkyō-to Shibuya-ku Dōgenzaka,Tōkyō-to Shibuya-ku Higashi,Tōkyō-to Shibuya-ku Jingūmae,Tōkyō-to Shibuya-ku Shibuya,Tōkyō-to Shinagawa-ku Higashigotanda,Tōkyō-to Shinagawa-ku Hiromachi,Tōkyō-to Shinjuku-ku Kabukichō,Tōkyō-to Shinjuku-ku Nishishinjuku,Tōkyō-to Suginami-ku Asagayaminami,Tōkyō-to Tachikawa-shi Izumichō,Tōkyō-to Taitō-ku Asakusa,Tōkyō-to Taitō-ku Higashiueno,Tōkyō-to Taitō-ku Kuramae,Tōkyō-to Toshima-ku Higashiikebukuro,Tōkyō-to Toshima-ku Mejiro,Tōkyō-to Toshima-ku Minamiikebukuro,Tōkyō-to Toshima-ku Nishiikebukuro,Tōkyō-to Toshima-ku Sugamo,Tōkyō-to Ōta-ku Kamata,Tōkyō-to Ōta-ku Kamiikedai,Ōsaka-fu Higashiōsaka-shi Aramotokita,Ōsaka-fu Neyagawa-shi Honmachi,Ōsaka-fu Sakai-shi Minamikawaramachi,Ōsaka-fu Suita-shi Izumichō,Ō

In [ ]:
lgb_paramsNN = {}
lgb_paramsNN['application'] = 'regression'
lgb_paramsNN['boosting'] = 'gbdt'
lgb_paramsNN['learning_rate'] = 0.015
lgb_paramsNN['num_leaves'] = 60
lgb_paramsNN['lambda_l2'] = 0.576
lgb_paramsNN['max_bin'] = 90
lgb_paramsNN['max_depth']= 12
lgb_paramsNN['min_split_gain'] = 0.014085456057661733
lgb_paramsNN['num_threads'] = 8
lgb_paramsNN['bagging_freq'] = 8
lgb_paramsNN['min_gain_to_split'] = 0
lgb_paramsNN['bagging_fraction'] = 0.87
lgb_paramsNN['feature_fraction'] = 0.74
lgb_paramsNN['num_threads'] = 4
lgb_paramsNN['metric'] = 'rmse'

In [ ]:
def do_train(X_train, X_valid, lgb_params, rounds):
    X_t = X_train.drop(['visitors'], axis=1)
    y_t = X_train['visitors'].values
    d_train = lgb.Dataset(X_t, y_t)
    X_v = X_valid.drop(['visitors'], axis=1)
    y_v = X_valid['visitors'].values
    d_valid = lgb.Dataset(X_v, y_v)
    watchlist = [d_train, d_valid]
    lgb_model = lgb.train(lgb_params, train_set=d_train, num_boost_round=rounds, 
                          valid_sets=watchlist, verbose_eval=1000, early_stopping_rounds = 300)
    test_pred = lgb_model.predict(X_v)
    rmsle = RMSLE(y_v, test_pred)
    print(X_t.columns)
    print(lgb_model.feature_importance())
    return rmsle, lgb_model

In [ ]:
X_train, X_valid = train_test_split(train_VW, test_size=0.3, random_state=74, shuffle=True)
rmsle, lgb_modelNN = do_train(X_train, X_valid, lgb_paramsNN, 12000)
print('Test RMSLE: %.3f' % rmsle)

Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 0.453186	valid_1's rmse: 0.480569
[2000]	training's rmse: 0.434831	valid_1's rmse: 0.477972
[3000]	training's rmse: 0.419711	valid_1's rmse: 0.476847
Early stopping, best iteration is:
[3656]	training's rmse: 0.410759	valid_1's rmse: 0.476539
Index(['air_store_id', 'latitude', 'longitude', 'holiday_eve', 'non_working',
       'genre_in_area', 'total_r_in_area', 'reserve_visitors', 'reserve_-12_h',
       'reserve_12_36_h',
       ...
       'Ōsaka-fu Ōsaka-shi Shinmachi', 'Ōsaka-fu Ōsaka-shi Ōgimachi',
       'Ōsaka-fu Ōsaka-shi Ōhiraki', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object', length=191)
[ 4075  2797  2202   964   417  1539  1913  4231   542   308   290   161
  1260  3748  3073  5003  1505  1435  2620   190  8043  4522  1964  2454
  7273  1427  1207  6790   364  1750   237  6294    60  1060  6597    10
  4360  1016  5426  2607  245

In [ ]:
sample_submission = pd.read_csv(root_path +'sample_submission.csv')
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
sample_submission['visit_date'] = sample_submission['id'].map(lambda x: str(x).split('_')[2])
test_id = sample_submission['id']

In [ ]:
test_id

0        air_00a91d42b08b08d9_2017-04-23
1        air_00a91d42b08b08d9_2017-04-24
2        air_00a91d42b08b08d9_2017-04-25
3        air_00a91d42b08b08d9_2017-04-26
4        air_00a91d42b08b08d9_2017-04-27
                      ...               
32014    air_fff68b929994bfbd_2017-05-27
32015    air_fff68b929994bfbd_2017-05-28
32016    air_fff68b929994bfbd_2017-05-29
32017    air_fff68b929994bfbd_2017-05-30
32018    air_fff68b929994bfbd_2017-05-31
Name: id, Length: 32019, dtype: object

In [ ]:

test_VW = test_VW.drop(columns=['visitors',"visit_date"], axis=1)
test_predNN = np.expm1(lgb_modelNN.predict(test_VW))
test_pred = test_predNN
result = pd.DataFrame({"id": test_id, "visitors": test_pred})   
result.to_csv(share_path+'LGB_sub.csv', index=False)
# files.download("LGB_sub.csv")
print('Done')

Done


In [ ]:
result.to_csv(share_path+'LGB_sub30VW.csv', index=False)